# Using Sherlock out-of-the-box
This notebook shows how to predict a semantic type for a given table column.
The steps are basically:
- Download files for word embedding and paragraph vector feature extraction (downloads only once) and initialize feature extraction models.
- Extract features from table columns.
- Initialize Sherlock.
- Make a prediction for the feature representation of the column.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [2]:
%env PYTHONHASHSEED

'13'

## Initialize feature extraction models

In [3]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:05.513540 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:04.191875 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.209930 seconds.


[nltk_data] Downloading package punkt to /Users/madelon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madelon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Extract features

In [4]:
data = pd.Series(
    [
        ["Jane Smith", "Lute Ahorn", "Anna James"],
        ["Amsterdam", "Haarlem", "Zwolle"],
        ["Chabot Street 19", "1200 fifth Avenue", "Binnenkant 22, 1011BH"]
    ],
    name="values"
)

In [5]:
data

0                 [Jane Smith, Lute Ahorn, Anna James]
1                         [Amsterdam, Haarlem, Zwolle]
2    [Chabot Street 19, 1200 fifth Avenue, Binnenka...
Name: values, dtype: object

In [8]:
extract_features(
    "../temporary.csv",
    data
)
feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)

Extracting Features: 100%|██████████| 3/3 [00:00<00:00, 167.51it/s]

Exporting 1588 column features


In [9]:
feature_vectors

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.116468,0.023982,-0.130867,0.006825,-0.135098,-0.070616,-0.052172,-0.067250,0.086256,-0.144385
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.054949,0.024502,-0.166001,-0.014375,-0.058199,0.009978,-0.046423,0.025163,0.036946,-0.086611
2,1.0,0.0,1.0,0.666667,0.0,2.0,1.0,3.0,-1.5,0.0,...,-0.022804,0.001741,0.047479,0.118293,-0.093435,0.036759,-0.004508,-0.087898,-0.117796,-0.191386


## Initialize Sherlock

In [11]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

## Predict semantic type for column

In [12]:
predicted_labels = model.predict(feature_vectors, "sherlock")

In [13]:
predicted_labels

array(['person', 'city', 'address'], dtype=object)